Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import resample

2025-01-30 22:33:47.123409: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-30 22:33:47.133015: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738256627.143861   11976 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738256627.147199   11976 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-30 22:33:47.159611: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Data handling

In [4]:
import pandas as pd

csv_file_path = '/home/prerna/landmark_detection/role_challenge_dataset_ground_truth.csv'

df = pd.read_csv(csv_file_path)

total_rows = len(df)

print(f"Total number of items (rows) in '{csv_file_path}': {total_rows}")


Total number of items (rows) in '/home/prerna/landmark_detection/role_challenge_dataset_ground_truth.csv': 622


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load dataset
file_path = "/home/prerna/landmark_detection/role_challenge_dataset_ground_truth.csv"  # Change this to your actual file path
data = pd.read_csv(file_path)

# Compute Euclidean distances for BPD and OFD
def euclidean_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

data["BPD"] = data.apply(lambda row: euclidean_distance(row["bpd_1_x"], row["bpd_1_y"], row["bpd_2_x"], row["bpd_2_y"]), axis=1)
data["OFD"] = data.apply(lambda row: euclidean_distance(row["ofd_1_x"], row["ofd_1_y"], row["ofd_2_x"], row["ofd_2_y"]), axis=1)

# # Define label: You can use BPD/OFD thresholds to classify into categories
# data["label"] = pd.qcut(data["BPD"], q=3, labels=["small", "medium", "large"])  # Example binning

print(data.head())


   image_name  ofd_1_x  ofd_1_y  ofd_2_x  ofd_2_y  bpd_1_x  bpd_1_y  bpd_2_x  \
0  000_HC.png      361       12      339      530      481       16      664   
1  001_HC.png      441      331      368      308      297      247      534   
2  002_HC.png      318      374      154      406      481      158      558   
3  003_HC.png      424      105      407      462      305      349      547   
4  004_HC.png      300      277      611      534       53      452      494   

   bpd_2_y         BPD         OFD  
0      318  353.118960  518.466971  
1      142  259.218055   76.537572  
2      215   95.801879  167.092789  
3      363  242.404620  357.404533  
4      308  463.914863  403.447642  


### Image Plotting


In [13]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt

# Load DataFrame from CSV (Modify this to your path)
csv_path = "/home/prerna/landmark_detection/train.csv"  # Change to your actual CSV file
data = pd.read_csv(csv_path)

def get_points_for_image(image_name):
    """
    Retrieves points from the DataFrame based on the image name.
    """
    row = data[data['image_name'] == image_name]
    if not row.empty:
        # Extract points for the image
        points = {
            "ofd_1": (row['ofd_1_x'].values[0], row['ofd_1_y'].values[0]),
            "ofd_2": (row['ofd_2_x'].values[0], row['ofd_2_y'].values[0]),
            "bpd_1": (row['bpd_1_x'].values[0], row['bpd_1_y'].values[0]),
            "bpd_2": (row['bpd_2_x'].values[0], row['bpd_2_y'].values[0])
        }
        return points
    return None

def plot_points_on_image(image_path, points, save_path=None):
    """
    Plots given points on the image and displays it.

    Parameters:
    - image_path: Path to the image
    - points: Dictionary with point labels and (x, y) coordinates
    - save_path: Path to save the plotted image (optional)
    """
    image = cv2.imread(image_path)
    
    if image is None:
        print(f"Error: Unable to load {image_path}")
        return
    
    # Colors for different point types
    colors = {
        "ofd_1": (255, 0, 0),  # Blue
        "ofd_2": (255, 0, 0),  # Blue
        "bpd_1": (0, 255, 0),  # Green
        "bpd_2": (0, 255, 0)   # Green
    }

    # Draw circles at given points
    for label, (x, y) in points.items():
        cv2.circle(image, (x, y), radius=5, color=colors[label], thickness=-1)
        cv2.putText(image, label, (x+5, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[label], 1)

    # Convert BGR to RGB for correct display in Matplotlib
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Show the image
    plt.figure(figsize=(6, 6))
    plt.imshow(image_rgb)
    plt.axis("off")
    plt.show()
    
    # Save the output image if needed
    if save_path:
        cv2.imwrite(save_path, image)
        print(f"Saved image with points at {save_path}")

# Iterate over the DataFrame and plot points
image_folder = r"/home/prerna/landmark_detection/filtered_images/"  # Update with your image folder path

for i, row in data.iterrows():
    if i < 2:
        image_name = "randomIntensity_495_HC.png"
        image_path = image_folder + image_name
        print(image_path)
        # Extract points as a dictionary
        points = get_points_for_image(image_name)
        
        plot_points_on_image(image_path, points)


/home/prerna/landmark_detection/filtered_images/randomIntensity_495_HC.png
Error: Unable to load /home/prerna/landmark_detection/filtered_images/randomIntensity_495_HC.png
/home/prerna/landmark_detection/filtered_images/randomIntensity_495_HC.png
Error: Unable to load /home/prerna/landmark_detection/filtered_images/randomIntensity_495_HC.png


[ WARN:0@3077.506] global loadsave.cpp:268 findDecoder imread_('/home/prerna/landmark_detection/filtered_images/randomIntensity_495_HC.png'): can't open/read file: check file path/integrity
[ WARN:0@3077.507] global loadsave.cpp:268 findDecoder imread_('/home/prerna/landmark_detection/filtered_images/randomIntensity_495_HC.png'): can't open/read file: check file path/integrity


Clahe Preprocessing

In [4]:

import cv2
import numpy as np
import os
from glob import glob

# Load image paths
image_dir = "/home/prerna/landmark_detection/selected_image"  # Change to your actual dataset folder
output_dir = "/home/prerna/landmark_detection/selected_image_clahe"
os.makedirs(output_dir, exist_ok=True)

# Create CLAHE object
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

# Preprocessing function for grayscale images
def preprocess_image(image_path):
    # Step 1: Load grayscale image
    grayscale = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale

    # Step 2: Convert to 8-bit unsigned integer (if not already)
    grayscale_8u = np.round(grayscale).astype(np.uint8)

    # Step 3: Apply CLAHE
    clahe_applied = clahe.apply(grayscale_8u)

    # Step 4: Convert to float32 (optional, if needed for further processing)
    clahe_float32 = clahe_applied.astype(np.float32)

    # # Step 5: Resize if needed (optional)
    # image_resized = cv2.resize(clahe_float32, (800, 540))  # Resize to standard size

    return clahe_float32
    
# Apply preprocessing to all images
image_paths = glob(os.path.join(image_dir, "*.png"))  # Adjust file extension as needed

for img_path in image_paths:
    preprocessed_img = preprocess_image(img_path)
    save_path = os.path.join(output_dir, os.path.basename(img_path))
    cv2.imwrite(save_path, preprocessed_img)

print("CLAHE preprocessing completed and saved in:", output_dir)

CLAHE preprocessing completed and saved in: /home/prerna/landmark_detection/selected_image_clahe
